## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

In [2]:
effect_size = sms.proportion_effectsize(0.2, 0.19)    # Розрахунок розміру ефекту на основі наших очікуваних показників

In [3]:
effect_size

0.025241594409087353

In [4]:
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )                                                  # Розрахунок необхідного розміру вибірки

required_n = ceil(required_n)                          # Округлення до наступного цілого числа

print(required_n)

24638


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [5]:
# Завантажуємо дані
df = pd.read_csv('../data/cookie_cats.csv', sep=None, engine='python')

# Виволимо розмір датасету
df.shape

(90189, 5)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [7]:
# Середнє retention_7 для різних версій гри

retension_mean = df.groupby('version')['retention_7'].mean()
retension_mean

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

<span style="color:green">**Спостереження: контрольна група gate_30 має кращі  показники утримання користувачів на 7 день.**</span>

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [12]:
session_counts = df['userid'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'Є {multi_users} користувачів, які зустрічаються кілька разів у наборі даних.')

Є 0 користувачів, які зустрічаються кілька разів у наборі даних.


In [13]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [14]:
control = df[df['version'] == 'gate_30'].copy()
treatment = df[df['version'] == 'gate_40'].copy()

In [15]:
ab_test = pd.concat([control, treatment], axis=0)
ab_test.reset_index(drop=True, inplace=True)

In [16]:
ab_test.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,1066,gate_30,0,False,False
3,2101,gate_30,0,False,False
4,2179,gate_30,39,True,False


In [17]:
ab_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [18]:
ab_test['version'].value_counts()

version
gate_40    45489
gate_30    44700
Name: count, dtype: int64

In [20]:
conversion_rates = ab_test.groupby('version')['retention_7']
conversion_rates = conversion_rates.agg(['mean', 'std', stats.sem])
conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']

conversion_rates.style.format('{:.5f}')

,conversion_rate,std_deviation,std_error
version,,,
gate_30,0.19020,0.39246,0.00186
gate_40,0.18200,0.38585,0.00181


In [26]:
control_results = ab_test[ab_test['version'] == 'gate_30']['retention_7']
treatment_results = ab_test[ab_test['version'] == 'gate_40']['retention_7']

In [27]:
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

In [28]:
successes

[8502, 8279]

In [31]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [35]:
z_stat, pval = proportions_ztest(successes, nobs=nobs)

lower_con, upper_con = proportion_confint(successes[0], nobs[0], alpha=0.05)
lower_treat, upper_treat = proportion_confint(successes[1], nobs[1], alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи control: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Довірчий інтервал 95% для групи treatment: [{lower_treat:.3f}, {upper_treat:.3f}]')

z statistic: 3.16
p-value: 0.002
Довірчий інтервал 95% для групи control: [0.187, 0.194]
Довірчий інтервал 95% для групи treatment: [0.178, 0.186]


Різниця між групами спостереження значуща, адже p-value = 0.002 < 0.05.

In [42]:
overlap = not (upper_con < lower_treat or upper_treat < lower_con)
if overlap:
    print("Інтервали перетинаються — різниця не є очевидною.")
else:
    print("Інтервали не перетинаються — різниця, ймовірно, є суттєвою.")

Інтервали не перетинаються — різниця, ймовірно, є суттєвою.


Інтервали не перетинаються, що підтрверджує те, що різниця є статистично значущою.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


H_0 = версія гри не впливає на утримання користувача </br>
H_1 = версія гри впливає на утримання користувача

In [39]:
pd.crosstab(ab_test['version'], ab_test['retention_7'])

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [40]:
crosstab = pd.crosstab(ab_test['version'], ab_test['retention_7'])

In [41]:
chi2, p, dof, expected = stats.chi2_contingency(crosstab)

print(f"χ² = {chi2:.3f}")
print(f"p-value = {p:.5f}")
print(f"Ступені свободи = {dof}")
print("Очікувані частоти:\n", expected)

χ² = 9.959
p-value = 0.00160
Ступені свободи = 1
Очікувані частоти:
 [[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]


Тест Хі-квадрат показав, що між зміною рівня і утриманням користувачів є зв'язок.